**2. Полносвязная нейронная сеть**

In [11]:
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from tensorflow.keras import optimizers
from keras.utils import np_utils
from keras.callbacks import TensorBoard

# Настраиваем параметры сети
NB_EPOCH = 50
BATCH_SIZE = 128
VERBOSE = 1 # Включаем индикатор выполнения
NB_CLASSES = 10 # Количество классов = количеству классов одежды
N_HIDDEN = 128 # Количество нейронов в скрытом слое
OPTIMIZER = optimizers.SGD() # Стохастический оптимизатор градиентного спуска
VALIDATION_SPLIT = 0.2 # Определяем какая часть обучающего набора зарезервирована для контроля качества
RESHAPED = 784 # Во входном слое с каждым пикселем изображения ассоциирован 1 нейрон, те. получается 28*28 = 784 нейрона

# Загружаем данные и разбиваем их на обучающий и тестовый набор
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Нормализуем данные
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000,RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

#Преобразовываем векторы классов в бинарные матрицы классов
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

model = Sequential() # Создаем модел
model.add(Dense(N_HIDDEN, input_shape = (RESHAPED,))) # Добавляем входной полносвязанный слой 
model.add(Activation('relu'))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax')) # Слой активации (Сигмоида)
model.summary()

#Компиляция модели
model.compile(loss = 'categorical_crossentropy', optimizer = OPTIMIZER, metrics = ['accuracy'])



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               100480    
                                                                 
 activation_9 (Activation)   (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 activation_10 (Activation)  (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                1290      
                                                                 
 activation_11 (Activation)  (None, 10)                0         
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-tr

In [12]:
# Подключаем модуль для построения графиков качества модели на валидационной и тренировочной выборке от количества эпох
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

# Обучаем нашу модель
model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs = NB_EPOCH, verbose = VERBOSE, validation_split = VALIDATION_SPLIT, callbacks=[tensorboard])

Epoch 1/50
375/375 [==============================] - 2s 5ms/step - loss: 1.2488 - accuracy: 0.6324 - val_loss: 0.8016 - val_accuracy: 0.7442
Epoch 2/50
375/375 [==============================] - 2s 4ms/step - loss: 0.7163 - accuracy: 0.7651 - val_loss: 0.6429 - val_accuracy: 0.7909
Epoch 3/50
375/375 [==============================] - 2s 4ms/step - loss: 0.6111 - accuracy: 0.7969 - val_loss: 0.5765 - val_accuracy: 0.8037
Epoch 4/50
375/375 [==============================] - 2s 4ms/step - loss: 0.5583 - accuracy: 0.8124 - val_loss: 0.5473 - val_accuracy: 0.8112
Epoch 5/50
375/375 [==============================] - 2s 4ms/step - loss: 0.5243 - accuracy: 0.8207 - val_loss: 0.5179 - val_accuracy: 0.8213
Epoch 6/50
375/375 [==============================] - 2s 4ms/step - loss: 0.5020 - accuracy: 0.8269 - val_loss: 0.4956 - val_accuracy: 0.8273
Epoch 7/50
375/375 [==============================] - 2s 4ms/step - loss: 0.4849 - accuracy: 0.8325 - val_loss: 0.4889 - val_accuracy: 0.8260
Epoch 

In [13]:
# Проверяем модель на тестовых данных
model.evaluate(X_test, Y_test, verbose = VERBOSE)

313/313 [==============================] - 0s 1ms/step - loss: 0.3766 - accuracy: 0.8681


[0.37664884328842163, 0.8680999875068665]

In [ ]:
# Строим графики качества модели на валидационной и тренировочной выборке от количества эпох
%load_ext tensorboard
%tensorboard --logdir logs

**Вывод** 

Качество модели по accuracy составило 88.81% на тренировочной выборке и 86.81% на тестовой

Как мы видим, качество классификации у модели с двумя скрытыми слоями на 2,46% выше, чем у модели без скрытых слоев. И при этом количество итераций уменьшилось с 200 до 50